In [1]:
!pip install faiss-cpu
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 73.8 MB/s eta 0:00:00


In [2]:
import math
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from torch.optim.adamw import AdamW
import random
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import os

import faiss
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer

In [3]:
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
PAD_TOKEN = "<pad>"
SEP_TOKEN = "<sep>"

In [4]:
!git clone https://github.com/brendenlake/SCAN.git

Cloning into 'SCAN'...
remote: Enumerating objects: 205, done.
remote: Total 205 (delta 0), reused 0 (delta 0), pack-reused 205 (from 1)
Receiving objects: 100% (205/205), 11.10 MiB | 18.13 MiB/s, done.
Resolving deltas: 100% (173/173), done.
Updating files: 100% (212/212), done.


In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
retriever_model = SentenceTransformer('all-MiniLM-L6-v2').to(DEVICE)
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(DEVICE)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
def load_scan_file(file_path):
    commands, actions = [], []
    with open(file_path, 'r') as f:
        for line in f:
            if "IN:" in line and "OUT:" in line:
                parts = line.strip().split(" OUT: ")
                commands.append(parts[0].replace("IN: ", ""))
                actions.append(parts[1])
    return commands, actions

In [ ]:
# --- FIX 1: Increase the tokenizer context window ---
# T5 can handle > 512 tokens due to relative attention, but needs a hint
tokenizer.model_max_length = 1024

def rag_predict(test_cmd, index, train_cmds, train_acts, k=3):
    # Retrieve k-neighbors
    query_vec = retriever_model.encode([test_cmd], convert_to_numpy=True)
    _, indices = index.search(query_vec.astype('float32'), k)

    # Build Prompt
    prompt = "Translate command to action sequence:\n"
    for i in indices[0]:
        prompt += f"Input: {train_cmds[i]} Output: {train_acts[i]}\n"
    prompt += f"Input: {test_cmd} Output:"

    # --- FIX 2: Add truncation and explicit max_length ---
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,        # Prevents error
        max_length=1024         # Allows longer context for RAG
    ).to(DEVICE)

    outputs = model.generate(**inputs, max_new_tokens=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 4. Main Experimental Loop
splits = [1, 2, 4, 8, 16, 64]
test_cmds, test_acts = load_scan_file("/content/SCAN/simple_split/tasks_test_simple.txt")

print(f"{'Split':<10} | {'Accuracy':<10}")
print("-" * 25)

for p in splits:
    # --- FIX 3: Corrected filename from tasks_test to tasks_train ---
    train_file = f"/content/SCAN/simple_split/size_variations/tasks_train_simple_p{p}.txt"

    if not os.path.exists(train_file):
        print(f"File not found: {train_file}")
        continue

    # Load and Index
    train_cmds, train_acts = load_scan_file(train_file)
    train_embs = retriever_model.encode(train_cmds, show_progress_bar=False)
    index = faiss.IndexFlatL2(train_embs.shape[1])
    index.add(np.array(train_embs).astype('float32'))

    # Evaluate
    correct = 0
    total = min(len(test_cmds), 500)
    for i in range(total):
        pred = rag_predict(test_cmds[i], index, train_cmds, train_acts)
        if pred.strip() == test_acts[i].strip():
            correct += 1

    accuracy = (correct / total) * 100
    print(f"p{p:<8} | {accuracy:>8.2f}%")

Split      | Accuracy  
-------------------------
